# Final Project: Huggingface Transformers를 이용한 한-영 번역

## 개요
- Assignment2의 연장선에서 한-영 번역 프로그램을 Huggingface의 Transformers와 Dataset을 이용해서 필요한 크기로 데이터를 줄이고 이를 모델에서 파인튜닝하여 번역 성능을 살펴보고 실제 예문의 번역을 출력
- [Huggingface NLP course의 7장 Translation](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt)을 근간으로 해서 (거의 그대로 활용할 수 있음) 구현할 수 있음
- Dataset을 자료를 받아서 필요한 크기로 나누고, 학습에 필요한 형태로 Dataset을 재구조화하고 tokenize하는 모듈을 구현
- 공개된 자료를 바탕으로 구현하기 때문에 성능보다는 전체 번역모듈을 Huggingface로 구현해보는 것을 주목표로 하기 때문에 완결성이 있어야 하며, 실제로 작동해야 함.
- FinalProject_학번_이름.ipynb
- Due 12월 8일 11시 59분

## 필요한 모듈 설치
- 프로그램 실행에 필요한 모듈, Huggingface, Dataset 등을 각자 알아서 설치

In [2]:
!pip install datasets # 2번 실행해야 잘 설치됨 (pip dependency issue)

In [3]:
!pip install transformers

In [4]:
!pip install accelerate -U

## Dataset
- Huggingface Hub에 있는 Dataset 중 `bongsoo/news_talk_en_ko` 는 한국어-영어뉴스 기사를 병렬로 작성한 130만 개의 데이터 셋이다.
- 이 데이터셋을 읽어서 colab에서 돌릴 수 있게, training, validation, test 데이터로 각각 120,000, 9,000, 1,000으로 줄여서 학습에 필요한 구조로 만듬
- 데이터를 자를때 순차적으로 자르지 말고 전체 데이터를 셔플한 후 필요한 크기로 자를 것
- 데이터셋을 pandas 형식으로 받은 후 할 수도 있고 여러 가능한 방법이 있음

In [5]:
from datasets import load_dataset, DatasetDict

In [6]:
raw_datasets = load_dataset("bongsoo/news_talk_en_ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/89.0 [00:00<?, ?B/s]

news_talk_en_ko_train_130000.tsv:   0%|          | 0.00/345M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1299999 [00:00<?, ? examples/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

In [8]:
shuffled_data = raw_datasets['train'].shuffle(seed=42) # seed는 42로 통일

In [9]:
TRAIN_SIZE = 120000
VALIDATION_SIZE = 9000
TEST_SIZE = 1000

train_data = shuffled_data.select(range(0, TRAIN_SIZE))
validation_data = shuffled_data.select(range(TRAIN_SIZE, TRAIN_SIZE + VALIDATION_SIZE))
test_data = shuffled_data.select(range(TRAIN_SIZE + VALIDATION_SIZE, TRAIN_SIZE + VALIDATION_SIZE + TEST_SIZE))

In [10]:
# 구조가 Skinner's 문장이 반복됨. 즉, 각 데이터에서 Skinner's 문장을 제외한 두 번째 문장들(value에 해당하는 값들)만 필요하다고 판단함 -> add_id_and_format 참고
print(train_data[0])
print(validation_data[0])
print(test_data[0])

{"Skinner's reward is mostly eye-watering.": "'Taegang Pale Ale' is a pale ale style beer brewed by The Booth paired up with Mikkeller, a global brewer, to change the prejudice that the Korean beer is less delicious than North Korea's Daedonggang beer.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.': '‘대강 페일에일’은 더부스가 한국 맥주가 북한 대동강 맥주보다 맛없다는 편견을 바꾸기 위해 글로벌 주류업체 미켈러와 만든 페일에일 스타일의 맥주다.'}
{"Skinner's reward is mostly eye-watering.": 'The city of Seoul began the reconstruction campaign in 2011 under the name of “Tourism Resource Recycling Project, Outside Seosomun,” recapturing the historicity of the area.', '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.': '서울시는 이 일대가 가진 역사성을 새롭게 조명해 ‘서소문 밖 역사유적지 관광자원화사업’이란 이름으로 2011년 재조성 대장정을 시작했다.'}
{"Skinner's reward is mostly eye-watering.": 'I got out of the house with my dog at three.', '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.': '나는 강아지를 데리고 3시에 집에서 나갔어요.'}


In [11]:
def add_id_and_format(example, idx):
    values = list(example.values())

    return {
        "id": idx,
        "translation": {
            "ko": values[1],
            "en": values[0]
        }
    }

In [12]:
train_data = train_data.map(lambda x, idx: add_id_and_format(x, idx), with_indices=True, remove_columns=train_data.column_names)
validation_data = validation_data.map(lambda x, idx: add_id_and_format(x, idx), with_indices=True, remove_columns=validation_data.column_names)
test_data = test_data.map(lambda x, idx: add_id_and_format(x, idx), with_indices=True, remove_columns=test_data.column_names)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
split_datasets = DatasetDict({
    "train": train_data,
    "validation": validation_data,
    "test": test_data
})

In [14]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 1000
    })
})

In [15]:
split_datasets["train"][1]["translation"]

{'en': 'Excuse me, have you come alone on this trip?',
 'ko': '실례지만 혹시 혼자 여행 오신 건가요?'}

## Huggingface
- 학습에 필요한 Huggingface 모델 사용
- AutoTokenizer, AutoModelForSeq2SeqLM 등을 사용
- 학습에 사용할 모델은 [T5](https://github.com/AIRC-KETI/ke-t5)("KETI-AIR/ke-t5-base")를 사용할 것

In [16]:
from transformers import pipeline

model_checkpoint = "KETI-AIR/ke-t5-base"
translator = pipeline("translation", model=model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [17]:
split_datasets["train"][172]["translation"]

{'en': 'He will always do the housework watching the television.',
 'ko': '그는 언제든 텔레비전을 보며 집안일을 할 것입니다.'}

In [18]:
# 기존 상태가 매우 좋지 않은 translator임을 알 수 있다.
print(translator("안녕"))
print(translator("그는 언제든 텔레비전을 보며 집안일을 할 것입니다."))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'translation_text': '큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브'}]
[{'translation_text': '큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브큐브'}]


## Tokenizer
- T5는 sentencepiece tokenizer를 사용하기 때문에 한-영 병렬 데이터의 자료를 학습시키기 위해서는 이 데이터를 tokenizer를 써서 프로세싱을 해야 한다. 이를 위한 모듈을 만들고 한국어, 영어데이터를 tokenize하여 모델에 입력할 수 있는 형태로(tokenized-dataset) 바꾼다
- 이를 위해서 Dataset의 map()을 활용하도록 한다.

In [19]:
from transformers import AutoTokenizer

model_checkpoint = "KETI-AIR/ke-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [20]:
ko_sentence = split_datasets["train"][1]["translation"]["ko"]
en_sentence = split_datasets["train"][1]["translation"]["en"]

inputs = tokenizer(ko_sentence, text_target=en_sentence)
inputs

{'input_ids': [54048, 525, 9297, 2700, 1747, 40970, 26869, 85, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [9665, 481, 7430, 612, 4, 84, 106, 941, 6059, 46, 110, 5975, 85, 1]}

In [21]:
wrong_targets = tokenizer(en_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Ex', 'c', 'use', '▁me', ',', '▁have', '▁you', '▁come', '▁alone', '▁on', '▁this', '▁trip', '?', '</s>']
['▁Ex', 'c', 'use', '▁me', ',', '▁have', '▁you', '▁come', '▁alone', '▁on', '▁this', '▁trip', '?', '</s>']


In [22]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex["ko"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [23]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Model
- 학습에 필요한 모델 설정

In [24]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Collator
- 학습할 자료를 정렬하고 모델에 배치 단위로 넘겨주기 위해 준비

In [25]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [26]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [27]:
batch["labels"]

tensor([[ 9665,   481,  7430,   612,     4,    84,   106,   941,  6059,    46,
           110,  5975,    85,     1],
        [   55, 35062,    17,     8,   166,  8913,    10,  9100,    10,    38,
             3,     1,  -100,  -100]])

In [28]:
batch["decoder_input_ids"]

tensor([[    0,  9665,   481,  7430,   612,     4,    84,   106,   941,  6059,
            46,   110,  5975,    85],
        [    0,    55, 35062,    17,     8,   166,  8913,    10,  9100,    10,
            38,     3,     1,     0]])

In [29]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[9665, 481, 7430, 612, 4, 84, 106, 941, 6059, 46, 110, 5975, 85, 1]
[55, 35062, 17, 8, 166, 8913, 10, 9100, 10, 38, 3, 1]


## Metric
- 학습한 모델을 측정할 매트릭을 준비
- 번역 모델에서는 주로 BLEU 점수를 사용
- BLEU 점수는 번역기가 생성한 문장이 레퍼런스(정답이라는 표현을 사용하지 않는 이유는 제대로 된 번역 문장이 오직 하나가 아니기 때문)문장과 얼마나 비슷한지 측정하는 점수

- nltk의 corpus_bleu로 bleu score를 계산함
- 이는 이후 test data를 가지고 bleu score 계산하는 데에 사용됨


In [30]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(reference, prediction, weights=[1, 0, 0, 0]):
    score = corpus_bleu(reference, prediction, weights=weights)
    return score

## 모델 학습(Train)
- 학습을 간단히 하기위해 허깅페이스에서 제공하는 Seq2SeqTrainer클래스와 학습 세부 조건은 Seq2SeqTrainingArguments를 활용할 수 있으나, 본 과제에서는 이를 쓰지 말고 Training를 직접 구현하도록 한다. Dataloader, Scheduler, ACCELERATOR, Optimizer 등을 설정하고 실제로 training loop를 돌려서 학습하고, evaluation 데이터로 성능을 검증
- colab에서 돌리기 위해서는 성능이 저하되겠지만, batch size 등을 적당하게 설정해야 함.

In [31]:
from accelerate import Accelerator
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm

In [32]:
accelerator = Accelerator()
device = accelerator.device

In [33]:
batch_size = 8
epochs = 3
learning_rate = 5e-5
gradient_accumulation_steps = 2

In [34]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], batch_size=batch_size, shuffle=True, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=batch_size, shuffle=False, collate_fn=data_collator
)

In [35]:
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
num_training_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)

In [38]:
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    model.train()
    train_loss = 0
    progress_bar = tqdm(train_dataloader, desc="Training", leave=False)

    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss / gradient_accumulation_steps
        accelerator.backward(loss)

        train_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        progress_bar.set_postfix({"Loss": loss.item()})

    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Training loss: {avg_train_loss:.4f}")

    model.eval()
    eval_loss = 0
    progress_bar = tqdm(eval_dataloader, desc="Validation", leave=False)

    with torch.no_grad():
        for batch in progress_bar:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            eval_loss += outputs.loss.item()

        avg_eval_loss = eval_loss / len(eval_dataloader)

        print(f"Validation loss: {avg_eval_loss:.4f}")
        print()

Epoch 1/3


Training:   0%|          | 0/15000 [00:00<?, ?it/s]

Training loss: 3.9016


Validation:   0%|          | 0/1125 [00:00<?, ?it/s]

Validation loss: 4.7779

Epoch 2/3


...

Validation loss: 3.2896

Epoch 3/3


Training:   0%|          | 0/15000 [00:00<?, ?it/s]

Training loss: 1.8484


Validation:   0%|          | 0/1125 [00:00<?, ?it/s]

Validation loss: 2.8318



In [39]:
# 런타임이 끊길 시 모델 유지를 위해 저장해놓기
torch.save(accelerator.unwrap_model(model).state_dict(), "final.pt")
print(f"Model saved to final.pt")

Model saved to final.pt


## 모델 테스트 (Test)
- 학습된 모델을 가지고 테스트 데이터로 테스트
- bleu score 계산

In [40]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=batch_size, shuffle=False, collate_fn=data_collator
)

In [41]:
model.eval()

test_loss = 0

progress_bar = tqdm(test_dataloader, desc="Testing", leave=False)
with torch.no_grad():
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        test_loss += outputs.loss.item()

avg_test_loss = test_loss / len(test_dataloader)

print(f"Test loss: {avg_test_loss:.4f}")

Testing:   0%|          | 0/125 [00:00<?, ?it/s]

Test loss: 2.8483


## Bleu Score

In [46]:
model.eval()
test_loss = 0
translated_sentences = []
reference_translations = []

progress_bar = tqdm(test_dataloader, desc="Testing", leave=False)

with torch.no_grad():
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model.generate(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        translated_sentences.extend(decoded_preds)

        labels = batch["labels"].clone()
        labels[labels == -100] = tokenizer.pad_token_id
        decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
        reference_translations.extend([[ref] for ref in decoded_refs])

Testing:   0%|          | 0/125 [00:00<?, ?it/s]

In [54]:
for i in range(min(5, len(translated_sentences))):
    print(f"Reference: {reference_translations[i][0]}")
    print(f"Prediction: {translated_sentences[i]}\n")

Reference: I got out of the house with my dog at three.
Prediction: Let's go to my house at 3 p.m.

Reference: Mayor Kwon was indicted for giving a speech about supporting candidate in elementary school alumni event and opening ceremony of Liberty Korea Party candidate on Aprill 22 and May 5 before June 13 local election.
Prediction: On April 22 and March 5, the Mayor Lee was arrested for saying the possibility of the candidate in the opening ceremony of the Democratic Party of Korea's candidates at a school club meeting and the opening ceremony of the Liberty Korea Party's candidates.

Reference: I'd like to send this letter registered.
Prediction: Let's send this letter to the ship.

Reference: It features a conscious focus on passes to connect with shooting, and fast ball processing using a teammate rather than dribbling.
Prediction: Let's see that they have actively actively actively focusing on the trick to connect the ball to the ball, rather than running the ball, and the speed 

In [58]:
bleu_score = calculate_bleu(reference_translations, translated_sentences)

print(f"BLEU Score: {bleu_score * 100:.2f}")

BLEU Score: 78.66


## Inference
- Assignment2에 쓰였던 문장들을 이 학습된 모델에서 그 결과를 살펴 보아라

- 모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.
- 미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?
- 은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔시세요
- 아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?
- 부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다
- 변기가 막혔습니다
- 그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?
- 비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.
- 속이 안좋을 때는 죽이나 미음으로 아침을 대신합니다
- 문대통령은 집단 이익에서 벗어나라고 말했다
- 이것 좀 먹어 볼 몇 일 간의 시간을 주세요
- 이 날 개미군단은 외인의 물량을 모두 받아 내었다
- 통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또 다른 꿈을 향해 나아간다
- 이번 구조 조정이 제품을 효과적으로 개발 하고 판매 하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다
- 요즘 이 프로그램 녹화하며 많은 걸 느낀다

In [51]:
sentences = [
    "모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.",
    "미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?",
    "은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔치세요.",
    "아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?",
    "부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다.",
    "변기가 막혔습니다.",
    "그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?",
    "비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.",
    "속이 안좋을 때는 죽이나 미음으로 아침을 대신합니다.",
    "문대통령은 집단 이익에서 벗어나라고 말했다.",
    "이것 좀 먹어 볼 몇 일 간의 시간을 주세요.",
    "이 날 개미군단은 외인의 물량을 모두 받아 내었다.",
    "통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또 다른 꿈을 향해 나아간다.",
    "이번 구조 조정이 제품을 효과적으로 개발하고 판매하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다.",
    "요즘 이 프로그램 녹화하며 많은 걸 느낀다."
]

In [52]:
translations = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", max_length=128, truncation=True).to(device)
    outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(translation)

In [53]:
for original, translated in zip(sentences, translations):
    print(f"Original: {original}")
    print(f"Translated: {translated}")
    print()

Original: 모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.
Translated: Let's put all water, sugar, and water in a one-foot-sized white lid.

Original: 미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?
Translated: We'm sorry, but if you change the ticket, can you change the ticket?

Original: 은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔치세요.
Translated: On the way to the bank, it's too far, so let's steal money.

Original: 아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?
Translated: Let's write a deposit of the stolen bag, so can you go to the office?

Original: 부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다.
Translated: Let's send 20 patients to Daegu when the number of HIV patients increased in Busan, so there is a lack of hospital facilities.

Original: 변기가 막혔습니다.
Translated: Let's get out of the toilet.

Original: 그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?
Translated: Let's see the 바지, which can I buy?

Original: 비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.
Translated: On the rain, I want to go to the retail store